### Getting started

##### Connection Setup:
   - Connect a Supernova with firmware version 4.0.0 or higher to a BMM350 Bosch target via the I3C HV bus. The Supernova will act as an I3C controller while the BMM350 as an I3C target.
   - Other I3C targets may be used, but you should check their static address and DAA allowed methods to properly test the device and understand the outcome of the evaluation.

##### Expected Messages and Notifications:
   - For each command sent to the I3C controller, the notebook should show the response returned by it.
   - Calling `i3cControllerGetTargetDevicesTable`, you can verify that the Target Devices Table is updated after each DAA method if run successfully: `i3cRSTDAA`, ``i3cSETDASA``, ``i3cSETAASA``, ``i3cENTDAA``, ``i3cControllerInitBus``, and `i3cControllerResetBus`
   
#####  Testing Process:
   - Run each cell in order.
   - Look for immediate feedback:
     - The response for each command directed to a Supernova.


### 1. Startup

#### 1.1 Import the python package

In [9]:
# Import Python package
import binhosupernova
from binhosupernova.supernova import Supernova
from binhosupernova.commands.i3c.definitions import *

#### 1.2 Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [10]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

#### 1.3 List all the Supernova devices connected to the PC host

The ``binhosupernova.getConnectedSupernovaDevicesList()`` gets a list of the Supernova devices plugged into the host PC machine.

In [11]:
binhosupernova.getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#7&9e2d326&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': '5F4BCE79166A34559DF3081DF5A8769D',
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'hardware_version': 'C',
  'firmware_version': '4.1.2'}]

#### 1.4 Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [12]:
# Create a device instance. One instance of the Supernova class represents a Supernova USB host adapter device.
supernova = Supernova()

#### 1.5 Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(serial, path)
```

- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``binhosupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.


In [13]:
# Use the method by default to connect to only one Supernova device.
supernova.open()

# Otherwise, use the path attribute to identify each Supernova device. Uncomment the line below and comment the line #2.
# supernova.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 1.6 Define and register a callback to handle responses and notifications from Supernova

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [14]:
# Define callback function
def callback_function(supernova_message: dict, system_message: dict) -> None:

    if supernova_message != None:

        # Print a header
        print(">> New message from SUPERNOVA:")
        print(supernova_message)

    if system_message != None:

        # Print a header
        print(">> New message from the SYSTEM:")
        print(system_message)

In [15]:
# Register callback function
supernova.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

### 2. I3C Protocol API

The I3C Protocol API methods are described below.

### 2.1 Configure the Supernova as an I3C Controller

``Supernova.i3cControllerInit(id, pushPullRate, i3cOpenDrainRate, i2cOpenDrainRate)``

This method initializes the Supernova as an I3C controller.

  - `id: int`: A 2-byte integer that represents the transfer ID.
  - `pushPullRate: I3cPushPullTransferRate`: Push-Pull frequency.
  - `i3cOpenDrainRate: I3cOpenDrainTransferRate`: I3C frequency in Open Drain mode.
  - `i2cOpenDrainRate: I2cTransferRate`: I2C frequency.

In [16]:
supernova.i3cControllerInit(getId(), I3cPushPullTransferRate.PUSH_PULL_3_125_MHZ_25_DC, I3cOpenDrainTransferRate.OPEN_DRAIN_1_MHZ, I2cTransferRate._400KHz)

{'module': 1, 'opcode': 0, 'message': 'I3C CONTROLLER INIT request success'}

>> New message from SUPERNOVA:
{'id': 1, 'command': 'I3C CONTROLLER INIT', 'result': 'SUCCESS'}


### 2.2 Set I3C voltage

``Supernova.setI3cVoltage(id, voltage_mV)``

This method supplies the indicated voltage to the I3C bus in mV, ranging from 800 mV up to 3300 mV. 

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``voltage_mV: c_int16 ``: The voltage parameter is a 2-byte integer in the range [800, 3300] mV and allows the value 0 mV to power off the output voltage.

There are 2 I3C ports:
- I3C LV: from 800 mV to 1199 mV
- I3C HV: from 1200 mV to 3300 mV

In [9]:
request_result = supernova.setI3cVoltage(getId(), 3300)

>> New message from SUPERNOVA:
{'id': 2, 'command': 'SYS SET I3C VOLTAGE', 'result': 'SUCCESS'}


### 2.3 I3C bus initialization

``Supernova.i3cControllerInitBus`` will execute a RSTDAA and then try to perform the DAA in the order specified by the MIPI Alliance: SETDASA, SETAASA and finally ENTDAA.
  - SETDASA will be executed if there is at least one target in the table argument for ``i3cControllerInitBus`` that allows SETDASA.
  - SETAASA will be executed if there is at least one target in the table argument for ``i3cControllerInitBus`` that allows SETAASA but doesn't allow SETDASA.
  - ENTDAA will be executed if the table is not full or if there is at least one target in the table argument for ``i3cControllerInitBus`` that allows ENTDAA but doesn't allow SETDASA nor SETAASA.

#### 2.3.1 Using ``i3cControllerInitBus`` with no target entries as argument

The CCCs involved are:
- RSTDAA
- ENTDAA since the Target Devices Table is empty when invoking i3cInitBus


In [10]:
# Get the Target Devices Table from the Supernova. It must be empty before the bus initialization.
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 3, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 0, 'table': []}


In [11]:
# Initialize and scan the I3C bus.
request_result = supernova.i3cControllerInitBus(getId())

>> New message from SUPERNOVA:
{'id': 4, 'command': 'I3C CONTROLLER INIT BUS', 'result': 'SUCCESS', 'invalid_addresses': []}


In [12]:
# Get the Target Devices Table from the Supernova
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 5, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 2, 'table': [{'static_address': 0, 'dynamic_address': 8, 'pid': [2, 8, 0, 112, 146, 11], 'bcr': 7, 'dcr': 68, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}, {'static_address': 0, 'dynamic_address': 9, 'pid': [7, 112, 16, 51, 0, 0], 'bcr': 38, 'dcr': 67, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_re

In [13]:
# Reset the bus before testing other initialization options.
request_result = supernova.i3cControllerResetBus(getId())

>> New message from SUPERNOVA:
{'id': 6, 'command': 'I3C CONTROLLER RESET BUS', 'result': 'SUCCESS'}


In [14]:
# Get the Target Devices Table from the Supernova
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 7, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 0, 'table': []}


#### 2.3.2 Using i3cInitBus with a target entry as argument

The CCCs involved are:
- RSTDAA 
- SETDASA since there is at least one entry that supports SETDASA 
- ENTDAA since the Target Devices Table is empty when invoking i3cInitBus


In [15]:
'''
If the static address of a target is known, sent it to the Supernova. The static address is mandatory
if it is expected to assign the dynamic address of the target through SETDASA or SETAASA CCCs.
'''
BMM350_STATIC_ADDRESS = 0x14

'''
Select the dynamic address to be assigned to the I3C Target if the DAA method is a SETDASA.
'''
BMM350_DYNAMIC_ADDRESS = 0x0C

"""
Set the features and the behavior of the I3C Target. For instance, in case that the I3C Target BCR[1] is
set, which means that the I3C Target support In-Band Interrupt (IBI) requests, define whether the Supernova
must accept or reject IBIs request from the target. Define also if the dynamic address will be assigned
using ENTDAA, SETDASA or SETAASA. To assign the dynamic address set above using ENTDAA, it is mandatory
to set also the BCR, DCR and PID in the table. If SETDASA or SETAASA are used, it is mandatory to set
the static address. It is also important to add here the I2C devices, setting the static address and the
target type as I2C_DEVICE.

In the example below, the SETDASA CCC is used to set the dynamic address set above to the target.
"""
BMM350_CONFIGURATION = {
        "targetType": TargetType.I3C_DEVICE,
        "IBIRequest": TargetInterruptRequest.ACCEPT_IBI,
        "CRRequest": ControllerRoleRequest.REJECT_CRR,
        "daaUseSETDASA": SetdasaConfiguration.USE_SETDASA,
        "daaUseSETAASA": SetaasaConfiguration.DO_NOT_USE_SETAASA,
        "daaUseENTDAA": EntdaaConfiguration.DO_NOT_USE_ENTDAA,
        "ibiTimestampEnable": IBiTimestamp.DISABLE_IBIT,
        "pendingReadCapability": PendingReadCapability.DISABLE_AUTOMATIC_READ
}

'''
Create the target devices table to be transferred to the Supernova. This table is a dictionary whose keys
are integer numbers starting from 0 and the maximum value possible is 10, which means that the maximum number
of I3C Targets is 11.
'''
table = {0: {'staticAddress': BMM350_STATIC_ADDRESS,
        'dynamicAddress': BMM350_DYNAMIC_ADDRESS,
        'bcr': 0x00,
        'dcr': 0x00,
        'pid': [0x00, 0x00, 0x00, 0x00, 0x00, 0x00],
        'configuration': BMM350_CONFIGURATION}}

'''
Initialize the bus using the predefined target devices table.
'''
request_result = supernova.i3cControllerInitBus(getId(), table)

>> New message from SUPERNOVA:
{'id': 8, 'command': 'I3C CONTROLLER INIT BUS', 'result': 'SUCCESS', 'invalid_addresses': []}


In [16]:
# Get the update target devices table.
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 9, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 2, 'table': [{'static_address': 20, 'dynamic_address': 12, 'pid': [0, 0, 0, 0, 0, 0], 'bcr': 0, 'dcr': 0, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}, {'static_address': 0, 'dynamic_address': 8, 'pid': [2, 8, 0, 112, 146, 11], 'bcr': 7, 'dcr': 68, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_c

#### 2.3.3 Using SETDASA CCC method directly

First, let's reset the bus.

`Supernova.i3cControllerResetBus(id)`

This method requests the Supernova to: 1. issue an RSTDAA CCC, 2. free the already assigned dynamic addresses and 3. clean up the target devices table. 

- `id: int`: A 2-byte integer that represents the transfer ID.

In [17]:
supernova.i3cControllerResetBus(getId())

{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER RESET BUS request success'}

>> New message from SUPERNOVA:
{'id': 10, 'command': 'I3C CONTROLLER RESET BUS', 'result': 'SUCCESS'}


In [18]:
# Get the update target devices table
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 11, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 0, 'table': []}


In [19]:
BMM350_STATIC_ADDRESS = 0x14
BMM350_DYNAMIC_ADDRESS = 0x0A

# Request a SETDASA
request_result = supernova.i3cSETDASA(getId(), BMM350_STATIC_ADDRESS, BMM350_DYNAMIC_ADDRESS)

>> New message from SUPERNOVA:
{'id': 12, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_SETDASA', 'payload_length': 1}


In [20]:
# Get the update target devices table
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 13, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 1, 'table': [{'static_address': 20, 'dynamic_address': 10, 'pid': [0, 0, 0, 0, 0, 0], 'bcr': 0, 'dcr': 0, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'REJECT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}]}


#### 2.3.4 Using SETAASA

In [21]:
supernova.i3cControllerResetBus(getId())

{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER RESET BUS request success'}

>> New message from SUPERNOVA:
{'id': 14, 'command': 'I3C CONTROLLER RESET BUS', 'result': 'SUCCESS'}


In [22]:
# Get the update target devices table
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 15, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 0, 'table': []}


In [23]:
BMM350_STATIC_ADDRESS = 0x14

# Request a SETAASA
request_result = supernova.i3cSETAASA(getId(), [BMM350_STATIC_ADDRESS])

>> New message from SUPERNOVA:
{'id': 16, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_SETAASA', 'payload_length': 1}


In [24]:
# Get the update target devices table
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 17, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 1, 'table': [{'static_address': 20, 'dynamic_address': 20, 'pid': [0, 0, 0, 0, 0, 0], 'bcr': 0, 'dcr': 0, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'REJECT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}]}


#### 2.3.5 Using ENTDAA

In [25]:
supernova.i3cControllerResetBus(getId())

{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER RESET BUS request success'}

>> New message from SUPERNOVA:
{'id': 18, 'command': 'I3C CONTROLLER RESET BUS', 'result': 'SUCCESS'}


In [26]:
# Get the update target devices table
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 19, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 0, 'table': []}


In [27]:
"""
If the static address of a target is known, sent it to the Supernova. The static address is mandatory
if it is expected to assign the dynamic address of the target through SETDASA or SETAASA CCCs.
"""
BMM350_STATIC_ADDRESS = 0x14

"""
Select the dynamic address to be assigned to the I3C Target if the DAA method is a SETDASA.
"""
BMM350_DYNAMIC_ADDRESS = 0x0C

"""
Set the features and the behavior of the I3C Target. For instance, in case that the I3C Target BCR[1] is
set, which means that the I3C Target support In-Band Interrupt (IBI) requests, define whether the Supernova
must accept or reject IBIs request from the target. Define also if the dynamic address will be assigned
using ENTDAA, SETDASA or SETAASA. To assign the dynamic address set above using ENTDAA, it is mandatory
to set also the BCR, DCR and PID in the table. If SETDASA or SETAASA are used, it is mandatory to set
the static address. It is also important to add here the I2C devices, setting the static address and the
target type as I2C_DEVICE.
In the example below, the ENTDAA CCC is used to set the dynamic address set above to the target.
"""
BMM350_CONFIGURATION = {
        "targetType": TargetType.I3C_DEVICE,
        "IBIRequest": TargetInterruptRequest.ACCEPT_IBI,
        "CRRequest": ControllerRoleRequest.REJECT_CRR,
        "daaUseSETDASA": SetdasaConfiguration.DO_NOT_USE_SETDASA,
        "daaUseSETAASA": SetaasaConfiguration.DO_NOT_USE_SETAASA,
        "daaUseENTDAA": EntdaaConfiguration.USE_ENTDAA,
        "ibiTimestampEnable": IBiTimestamp.DISABLE_IBIT,
        "pendingReadCapability": PendingReadCapability.DISABLE_AUTOMATIC_READ
}

"""
Create the target devices table to be transferred to the Supernova. This table is a dictionary whose keys
are integer numbers starting from 0 and the maximum value possible is 10, which means that the maximum number
of I3C Targets is 11.
"""
table = {0: {'staticAddress': BMM350_STATIC_ADDRESS,
        'dynamicAddress': BMM350_DYNAMIC_ADDRESS,
        'bcr': 0x00,
        'dcr': 0x00,
        'pid': [0x00, 0x00, 0x00, 0x00, 0x00, 0x00],
        'configuration': BMM350_CONFIGURATION}}

# Initialize and scan the I3C bus.
request_result = supernova.i3cENTDAA(getId(), table)

>> New message from SUPERNOVA:
{'id': 20, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_ENTDAA', 'payload_length': 0}


In [28]:
# Get the update target devices table.
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 21, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 2, 'table': [{'static_address': 0, 'dynamic_address': 8, 'pid': [2, 8, 0, 112, 146, 11], 'bcr': 7, 'dcr': 68, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}, {'static_address': 0, 'dynamic_address': 9, 'pid': [7, 112, 16, 51, 0, 0], 'bcr': 38, 'dcr': 67, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_r

#### 2.3.6 Change the dynamic address using SETNEWDA

In [29]:
OLD_DYNAMIC_ADDRESS = 0x08
NEW_DYNAMIC_ADDRESS = 0x0F

request_result = supernova.i3cSETNEWDA(getId(), OLD_DYNAMIC_ADDRESS, NEW_DYNAMIC_ADDRESS)

>> New message from SUPERNOVA:
{'id': 22, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_SETNEWDA', 'payload_length': 1}


In [30]:
# Get the update target devices table
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 23, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 2, 'table': [{'static_address': 0, 'dynamic_address': 15, 'pid': [2, 8, 0, 112, 146, 11], 'bcr': 7, 'dcr': 68, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}, {'static_address': 0, 'dynamic_address': 9, 'pid': [7, 112, 16, 51, 0, 0], 'bcr': 38, 'dcr': 67, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_

In [31]:
supernova.i3cControllerResetBus(getId())

{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER RESET BUS request success'}

>> New message from SUPERNOVA:
{'id': 24, 'command': 'I3C CONTROLLER RESET BUS', 'result': 'SUCCESS'}


### 3. Close communication

Use the ``Supernova.close()`` method to end the communication with the Supernova device and release the used memory in the background like threads and so on.

In [17]:
# Close the communication with the Supernova device.
supernova.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}